<a href="https://colab.research.google.com/github/LucasSilvaFerreira/BioNodes/blob/master/BioNodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import re 
from rich import print
from rich.columns import Columns
from rich.panel import Panel
from bokeh.plotting import figure
from bokeh.io import output_notebook, show 
from bokeh.models import Label




In [6]:
!pip install rich

In [60]:
cmd_txt = '''tophat --solexa-quals -g 2 -p 8 --library-type fr-unstranded -j
annotation/Danio_rerio.Zv9.66.spliceSites -o tophat/ZV9_2cells
genome/ZV9 data/2cells_1.fastq data/2cells_2.fastq'''

In [61]:
cmd_txt = re.sub(' |\t|\n+', ' ', cmd_txt)
cmd_txt

'tophat --solexa-quals -g 2 -p 8 --library-type fr-unstranded -j annotation/Danio_rerio.Zv9.66.spliceSites -o tophat/ZV9_2cells genome/ZV9 data/2cells_1.fastq data/2cells_2.fastq'

In [62]:
# output_notebook()
# def plot_node(list_node):
#   top = [(n+1.3) -0.1 for n, b in enumerate(list_node)]
#   bot = [(n+0.8)  for n, b in enumerate(list_node)]  
#   p = figure(plot_width=400, plot_height=400)
#   p.quad(top=top, bottom=bot, left=[1, 1, 1],
#           right=[3, 3, 3], color=['#787878', '#787878', '#787878'],fill_alpha=0.3,line_width=2, )

#   for e, b in enumerate(list_node):
      
#       p.add_layout(Label(x=1.7, y=e+1, text=b.__str__()))
#   p.axis.visible = False
#   show(p)

In [294]:
from enum import Enum

class FileType(Enum):
    TXT=0
    BAM=1


class Slot():
  def __init__(self, name, slot_type, description='No Description'):
    self.name= name
    assert isinstance(slot_type, Enum), 'slot_type Should be a FileType'
    self.slot_type = slot_type
    self.description = description

  def __repr__(self):
    out_repr = f"{self.name} \n {self.slot_type} \n {self.description}"
    return out_repr

  


  


class Tool():
  def __init__(self, name):

    self.name = name
    self.in_slots     = {}
    self.out_slots    = {}
    self.add_property = {}

  def add_in_slot(self, slot):
    assert slot.name not in self.in_slots,  f'This  slot.name : {slot.name}  already exists!'
    self.in_slots[slot.name] = slot

      

  def add_out_slot(self, slot):
      assert slot.name not in  self.out_slots, f'This  slot.name : {slot.name}  already exists!'
      self.out_slots[slot.name] = slot

  def get_in_slot_by_name(self, name):
         return self.in_slots[name]

  def get_out_slot_by_name(self, name):
         return self.out_slots[name]

  def __get_all_slots(self, slot_type):
    if slot_type =='in':
        slot_to_get = self.in_slots
        t_tring = 'INPUT'
    if slot_type =='out':
        slot_to_get = self.out_slots
        t_tring = 'OUTPUT'

    final_string = []
    final_string.append(f'       <<{t_tring} SLOTS>>')
    for k, s in slot_to_get.items():
      final_string.append('-'* 30)
      final_string.append(f"{s}")
    final_string.append('-'* 30)
    return final_string
  
  def show_slots_in(self):
    print ('\n'.join(self.__get_all_slots("in")))

  def show_slots_out(self):
    print ('\n'.join(self.__get_all_slots("out")))


  def connect(self, this_out, input_object, input_slot_name):
    bind_out = self.get_in_slot_by_name(this_out)
    bind_in = input_object.get_out_slot_by_name(input_slot_name)
    assert bind_out.slot_type == bind_in.slot_type, f'{bind_out.slot_type} > is not compatible with >  {bind_in.slot_type}'
    return True

class File_input(Tool):
  def __init__(self, name, file_in):
        super().__init__(name)
        self.file_in = file_in
        self.add_out_slot(Slot('File', self.get_extension() )) 

  def __repr__(self):
      self.show_slots_out()
      return self.name



  def get_extension(self):
      try: 
          return FileType[ self.file_in.split('.')[-1].upper()]
      except KeyError:
          print (f"Cant recognize the extension .{self.file_in.split('.')[-1].upper()}")








In [295]:

test  = Tool('Teste')
test.add_in_slot(Slot('Txt_raw',    FileType.TXT))
test.add_in_slot(Slot('Bam_file', FileType.BAM))
test.add_out_slot(Slot('Stranded_bam', FileType.BAM))
test.show_slots_in()
test.show_slots_out()

       <<INPUT SLOTS>>
------------------------------
Txt_raw 
 FileType.TXT 
 No Description
------------------------------
Bam_file 
 FileType.BAM 
 No Description
------------------------------
       <<OUTPUT SLOTS>>
------------------------------
Stranded_bam 
 FileType.BAM 
 No Description
------------------------------


In [298]:
f1 = File_input('teste_txt', 'teste/files/teste.txt') 
f1.show_slots_out()
f2 = File_input('bam_test', 'teste/files/teste.bam')



test.connect('Txt_raw', f1, 'File')


       <<OUTPUT SLOTS>>
------------------------------
File 
 FileType.TXT 
 No Description
------------------------------


True

In [300]:
test.connect('Bam_file', f1, 'File')

AssertionError: ignored

In [140]:
\dir(FileType)

['BAM', 'TXT', '__class__', '__doc__', '__members__', '__module__']

<enum 'FileType'>